<a href="https://colab.research.google.com/github/git-pi-e/voicebot_backend/blob/main/Customer_Success_Voicebot_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend for Voicebot

## Instructions
- Add your ngrok token as well as HuggingFace token to Google Colab secrets.
- Run all the cells.
- Pick up the ngrok Public URL and use it as the backend url for the deployed frontend (environment variable) or local server (.env.local -> NEXT_PUBLIC_BACKEND_URL)

In [3]:
!pip install fastapi uvicorn transformers
!pip install nest-asyncio pyngrok
!pip install python-multipart
!pip install soundfile pydub
!pip install torch
!pip install gTTS # Alternative

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolve

In [ ]:
from fastapi import FastAPI, UploadFile, File, Body, HTTPException
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
from transformers import pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer
from pyngrok import ngrok
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from gtts import gTTS
from google.colab import userdata
import soundfile as sf
import io
import requests
import uvicorn
import nest_asyncio
import logging
import base64
import os

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize FastAPI
app = FastAPI()

# Enable CORS
origins = ["*"]  # Adjust this as needed
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load models
try:
    stt_model = pipeline("automatic-speech-recognition", model="openai/whisper-tiny.en")
    logger.info("STT model loaded successfully")
except Exception as e:
    logger.error(f"Error loading STT model: {e}")
    stt_model = None

try:
    llm_model = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    logger.info("LLM model loaded successfully")
except Exception as e:
    logger.error(f"Error loading LLM model: {e}")
    llm_model = None

# try:
#     # Load the TTS model
#     tts_tokenizer = T5Tokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")
#     tts_model = T5ForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1")
#     logger.info("TTS model loaded successfully")
# except Exception as e:
#     logger.error(f"Error loading TTS model: {e}")
#     tts_model = None

# Utility function to create a retryable session
def create_retryable_session(retries=3, backoff_factor=0.3, status_forcelist=(500, 502, 504)):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# Test route to ensure the API is running
@app.get("/")
async def root():
    logger.info("Root endpoint accessed")
    return {"message": "Welcome to the Colab-hosted Voice Interaction API"}

# Transcription route with detailed error handling
@app.post("/transcribe")
async def transcribe_audio(file: UploadFile = File(...)):
    if stt_model is None:
        logger.error("STT model not available")
        raise HTTPException(status_code=500, detail="STT model not available")

    try:
        logger.info("Starting transcription process...")
        logger.info(f"Received file: {file.filename}, Content Type: {file.content_type}")

        # Read the WAV file content
        audio_content = await file.read()
        logger.info(f"Audio content size: {len(audio_content)} bytes")

        # Ensure the audio content is a valid WAV file
        try:
            # Use SoundFile to check the file and read data
            wav_io = io.BytesIO(audio_content)
            wav_io.seek(0)
            data, samplerate = sf.read(wav_io)
        except Exception as file_error:
            logger.error(f"Error reading WAV file: {file_error}")
            raise HTTPException(status_code=400, detail="Invalid WAV file format. Please upload a valid WAV file.")

        # Transcribe audio using the loaded model
        try:
            transcript = stt_model(audio_content)["text"]
            logger.info(f"Transcription result: {transcript}")
            print(f"Transcription result: {transcript}")
            logger.info("Transcription successful")


            return {"transcript": transcript}
        except ValueError as model_error:
            logger.error(f"Model inference error: {model_error}")
            raise HTTPException(status_code=500, detail="Error during transcription. The audio may be malformed or corrupted.")
        except Exception as unknown_model_error:
            logger.error(f"Unexpected model error: {unknown_model_error}", exc_info=True)
            raise HTTPException(status_code=500, detail="An unexpected error occurred during transcription.")

    except Exception as e:
        logger.error(f"Unhandled exception: {e}", exc_info=True)
        raise HTTPException(status_code=500, detail=f"Error transcribing audio: {e}")

# Text generation route with detailed error handling
@app.post("/generate")
async def generate_response(data: dict = Body(...)):

    prompt = data.get('prompt')
    if not prompt:
        raise HTTPException(status_code=400, detail="Prompt is required")

    if llm_model is None:
        logger.error("LLM model not available")
        raise HTTPException(status_code=500, detail="LLM model not available")

    try:
        logger.info(f"Generating response for prompt: {prompt}")
        print(f"Generating response for prompt: {prompt}")

        response = llm_model(prompt, max_length=100, num_return_sequences=1)

        if not response or "generated_text" not in response[0]:
            logger.error("Invalid response format from LLM model")
            raise HTTPException(status_code=502, detail="Failed to generate valid response")

        response_text = response[0]['generated_text']

        # Remove the prompt from the generated response
        response_text = response_text.replace(prompt, "").strip()

        logger.info(f"Generated response: {response_text}")
        print(f"Generated response: {response_text}")
        logger.info("Response generation successful")

        # Generate audio file using gTTS
        tts = gTTS(text=response_text, lang='en')

        # Use a unique identifier for the file name
        filename = "response_audio.mp3"
        tts.save(filename)
        logger.info("Text-to-speech conversion successful")
        print("Text-to-speech conversion successful")

        # Option 1: Return base64 encoded audio in response
        with open(filename, "rb") as audio_file:
            audio_bytes = audio_file.read()
            audio_base64 = base64.b64encode(audio_bytes).decode("utf-8")

        # Clean up the file after sending the response
        os.remove(filename)

        return {
            "response": response_text,
            "audio_base64": audio_base64
        }

    except requests.exceptions.RequestException as re:
        logger.error(f"RequestException: {re}")
        if isinstance(re, requests.exceptions.ConnectionError):
            error_message = "Connection to the LLM server failed. Please try again later."
        elif isinstance(re, requests.exceptions.Timeout):
            error_message = "The request to the LLM server timed out. Please try again."
        else:
            error_message = "An error occurred while communicating with the LLM server."
        raise HTTPException(status_code=503, detail=error_message)

    except Exception as e:
        logger.error(f"Unhandled exception during response generation: {e}", exc_info=True)
        raise HTTPException(status_code=500, detail=f"Error generating response: {e}")

# Speech synthesis route with detailed error handling
# @app.post("/synthesize")
# async def synthesize_speech(text: str = Body(...)):
#     if tts_model is None:
#         logger.error("TTS model not available")
#         raise HTTPException(status_code=500, detail="TTS model not available")
#     try:
#         logger.info(f"Synthesizing speech for text: {text}")

#         # Tokenize and generate speech using the TTS model
#         input_ids = tts_tokenizer.encode(text, return_tensors="pt")
#         output = tts_model.generate(input_ids)
#         audio_output = output[0].numpy()

#         if audio_output is None:
#             logger.error("Failed to generate audio output")
#             raise HTTPException(status_code=502, detail="Failed to synthesize audio")

#         logger.info("Speech synthesis successful")

#         # Stream the audio output
#         return StreamingResponse(io.BytesIO(audio_output), media_type="audio/wav")

#     except requests.exceptions.RequestException as re:
#         logger.error(f"RequestException: {re}")
#         if isinstance(re, requests.exceptions.ConnectionError):
#             error_message = "Connection to the TTS server failed. Please try again later."
#         elif isinstance(re, requests.exceptions.Timeout):
#             error_message = "The request to the TTS server timed out. Please try again."
#         else:
#             error_message = "An error occurred while communicating with the TTS server."
#         raise HTTPException(status_code=503, detail=error_message)
#     except Exception as e:
#         logger.error(f"Unhandled exception during speech synthesis: {e}", exc_info=True)
#         raise HTTPException(status_code=500, detail=f"Error synthesizing speech: {e}")

# Configure ngrok with your authtoken
ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))

# This allows the FastAPI app to run within the notebook
nest_asyncio.apply()

# Expose the app using ngrok
try:
    public_url = ngrok.connect(8000)
    logger.info(f"Public URL: {public_url}")
    logger.info("Colab-hosted Voice Interaction API is now running!")
    print(f"Public URL: {public_url}")
except Exception as e:
    logger.error(f"Error connecting to ngrok: {e}")

# Run the app
uvicorn.run(app, host="0.0.0.0", port=8000)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-28' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

Public URL: NgrokTunnel: "https://7fff-35-236-220-121.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [479]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription result:  Hello there.
INFO:     49.207.235.88:0 - "POST /transcribe HTTP/1.1" 200 OK
INFO:     49.207.235.88:0 - "OPTIONS /generate HTTP/1.1" 200 OK


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Transcription result:  It will hurt.
INFO:     49.207.235.88:0 - "POST /transcribe HTTP/1.1" 200 OK
Generating response for prompt:  It will hurt.
Generated response: JASON: (sighs) I know. But I can't let it control me.

SARAH: (smiling) You're not alone. We're here for you.

JASON: (smiling back) Thanks, Sarah.

SARAH: (nodding) We're here for you, Jason.

JASON: (smiling) I
Text-to-speech conversion successful
INFO:     49.207.235.88:0 - "POST /generate HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription result:  Okay.
INFO:     49.207.235.88:0 - "POST /transcribe HTTP/1.1" 200 OK
INFO:     49.207.235.88:0 - "OPTIONS /generate HTTP/1.1" 200 OK
Generating response for prompt:  Okay.
Generated response: So, let's say I have a list of strings, and I want to find the longest common substring between each pair of strings in the list.  How would I go about doing that?
Text-to-speech conversion successful
INFO:     49.207.235.88:0 - "POST /generate HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription result:  Are you sure about that?
INFO:     49.207.235.88:0 - "POST /transcribe HTTP/1.1" 200 OK
Generating response for prompt:  Are you sure about that?
Generated response: I mean, I've never been to a place like this before. It's so different from anything I've ever seen before.

JASON: (smiling) Yeah, it's definitely different. But I'm excited to explore and learn about this place.

SARAH: (nodding) Yeah, I'm excited too. I've always wanted to see something like this.
Text-to-speech conversion successful
INFO:     49.207.235.88:0 - "POST /generate HTTP/1.1" 200 OK
